In [1]:
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import numpy as np
import pandas as pd
import random

In [ ]:
test_df = pd.read_csv("./out/test_reshoot_preadjustion.csv")

In [ ]:
test_df.head(15)

In [ ]:
#cnt      = 0
gap_list = []
for idx, row in test_df.iterrows():
    timestamp = row['TIMESTAMP_BK']
    if (pd.isna(timestamp)):
        prev_timestamp = test_df.iloc[idx-1]['TIMESTAMP_BK']
        if (not pd.isna(prev_timestamp)):
            gap_start = idx-1
            pass
        next_timestamp = test_df.iloc[idx+1]['TIMESTAMP_BK']
        if (not pd.isna(next_timestamp)):
            gap_end = idx+1
            gap_list.append((gap_start, gap_end))
            
            #cnt = cnt + 1
            #if cnt>5:
            #    break
            
len(gap_list)
        

In [ ]:
gap_list[:5]

In [ ]:
#cnt = 0
for gap in gap_list:
    gap_start     = gap[0]
    gap_end       = gap[1]
    scale_min     = test_df.iloc[gap_start]['TIMESTAMP_BK']
    scale_max     = test_df.iloc[gap_end  ]['TIMESTAMP_BK']
    bk_timestamps = test_df.iloc[gap_start:gap_end+1]['TIMESTAMP'].values
    
    rescaled = np.interp(
        bk_timestamps, 
        (bk_timestamps.min(), bk_timestamps.max()), 
        (scale_min, scale_max)
    )
    iter_count = gap_end - gap_start
    for i in range(0, iter_count+1):
        idx = gap_start+i
        test_df.at[idx, 'TIMESTAMP_SCALED'] = rescaled[i]
        test_df.at[idx, 'DATETIME_SCALED' ] = datetime.utcfromtimestamp(rescaled[i])
        
    #cnt = cnt+1
    #if cnt>5:
    #    break

In [ ]:
for idx, row in test_df.iterrows():
    timestamp_scaled = row['TIMESTAMP_SCALED']
    if (pd.isna(timestamp_scaled)):
        test_df.at[idx, 'TIMESTAMP_SCALED'] = row['TIMESTAMP']

In [ ]:
test_df.to_csv("./out/test_df_timestamp_adjusted.csv")
test_df[['index', 'TIMESTAMP_SCALED']].to_csv("./out/output_ready.csv", index=False)

In [ ]:
test_df.head(30)